In [1]:
import theano

Using cuDNN version 5105 on context None
Mapped name None to device cuda: GeForce GT 740M (0000:01:00.0)


In [72]:
from theano import gradient,function
import numpy as np
import theano.tensor as T
from sklearn import mixture

class GMMGradOp(theano.Op):
    def __init__(self,gm_num, ndim):
        super(GMMGradOp,self).__init__()  
        self.otypes = [T.fmatrix]
        self.itypes = [T.fvector]   
        self.ndim = ndim
        self.gm_num = gm_num
        self.reg_coef = 1e-5
        Yvec = T.dvector('Y')
        meansvec = T.dvector('means')
        covarsvec = T.dvector('covars')
        weights = T.dvector('weights')
        lam = T.dscalar('lambda')
        N,M = self.build_linear_system(Yvec,meansvec, covarsvec, weights, lam)
        self.NM = function([Yvec, meansvec, covarsvec, weights, lam], [N,M], allow_input_downcast=True)
        
    def build_lagrangian(self,Yvec,meansvec,covarsvec,weights,lam):
        n_dim = self.ndim
        gm_num = self.gm_num
        Y = Yvec.reshape((-1,n_dim))
        means = T.reshape(meansvec, (gm_num, n_dim))
        covars = T.reshape(covarsvec, (gm_num,n_dim))
        log_prob = calc_log_prob_gauss_vector(Y, means,covars,weights)        
        return T.sum(log_prob) + lam * (T.sum(weights) - 1) 
        
    def build_linear_system(self,Yvec,meansvec,covarsvec,weights,lam):
        n_dim = self.ndim
        gm_num = self.gm_num
        lagrangian = self.build_lagrangian(Yvec,meansvec,covarsvec,weights,lam)
        d_lagrangian = gradient.jacobian(lagrangian, [meansvec, covarsvec, weights])
               
        hm = gradient.jacobian(d_lagrangian[0], [Yvec, meansvec, covarsvec, weights])
        hc = gradient.jacobian(d_lagrangian[1], [Yvec, meansvec, covarsvec, weights])
        hw = gradient.jacobian(d_lagrangian[2], [Yvec, meansvec, covarsvec, weights, lam])
        
        mean_row = T.concatenate((hm[1], hm[2], hm[3], T.zeros((n_dim*gm_num, 1))), axis=1)
        cov_row = T.concatenate((hc[1], hc[2], hc[3], T.zeros((n_dim*gm_num, 1))), axis=1)
        
        weight_row = T.concatenate((hw[1], hw[2], hw[3], T.reshape(hw[4], (gm_num, 1))), axis=1)
        lambda_row = T.concatenate(
            (T.zeros((1, 2*n_dim*gm_num)),
             T.reshape(hw[4], (1, gm_num)),
             T.zeros((1, 1))), axis=1)
        M = T.concatenate((mean_row, cov_row, weight_row, lambda_row))
        N = T.concatenate((-hm[0], -hc[0], -hw[0], T.zeros((1, hw[0].shape[1]))))
        return N,M#MX = N
    
    def solve_diag_linear(self,N,M,a,b,c):#MX=N
        D = M[2 * par_dim:, 2 * par_dim:]
        a = a + T.ones(a.shape[0])*self.reg_coef
        c = c + T.ones(a.shape[0]) * self.reg_coef
        D = D + T.eye(D.shape[0]) * self.reg_coef
        Di = T.nlinalg.matrix_inverse(D)
        e = 1. / (a - b / c * b)
        f = -e * b / c
        h = (np.ones(a.shape[0]) - f * b) / c
        dX = np.zeros(N.shape)
        for i in range(0, n_samples):
            N1 = N[:, i * n_dim:(i + 1) * n_dim]
            for gi in range(0, gm_num):
                n_mu_gi = np.diag(N1[gi * n_dim:(gi + 1) * n_dim, 0:n_dim])
                e_gi = e[gi * n_dim:(gi + 1) * n_dim]
                n_cov_gi = np.diag(
                    N1[n_dim * gm_num + gi * n_dim:n_dim * gm_num + (gi + 1) * n_dim, 0:n_dim])
                f_gi = f[gi * n_dim:(gi + 1) * n_dim]
                h_gi = h[gi * n_dim:(gi + 1) * n_dim]
                dX[gi * n_dim: (gi + 1) * n_dim, i * n_dim:(i + 1) * n_dim] = np.diag(
                    e_gi * n_mu_gi + f_gi * n_cov_gi)
                dX[n_dim * gm_num + gi * n_dim: n_dim * gm_num + (gi + 1) * n_dim,
                i * n_dim: (i + 1) * n_dim] = np.diag(f_gi * n_mu_gi + h_gi * n_cov_gi)
        dX[n_dim * 2 * gm_num:, :] = Di.dot(N[n_dim * 2 * gm_num:, :])
        return dX
        
    def solve_general_linear(self,N,M):#MX=N
        M = M + self.reg_coef * np.eye(M.shape[0])
        return T.slinalg.solve(M,N)
    
    def solve_linear_system(self,N,M,par_dim):
        def diag(M):            
            a = T.diag(M)
            A = T.diag(a)
            return a,T.abs(A - M).sum()
        a,na = diag(M[0:par_dim, 0:par_dim])
        b,nb = diag(M[0:par_dim, par_dim:2 * par_dim])
        c,nc = diag(M[par_dim:2 * par_dim, par_dim:2 * par_dim])
        return theano.ifelse.ifelse(T.le(na+nb+nc,1e-6),\
                             self.solve_diag_linear(N,M,a,b,c),\
                             self.solve_general_linear(N,M))
        return dX
        
        
        
        
        
    
    def solve_lin_sys_for_gmm(self, Xvec, meansvec, covarsvec, weights):
        gm_num = len(weights)
        n_dim = self.ndim
        n_samples = len(Xvec)//n_dim
        lam = n_samples
        NM = self.NM(Xvec, meansvec, covarsvec, weights,lam)
        N = NM[0]
        M = NM[1]
        par_dim = gm_num * n_dim
        a = np.diag(M[0:par_dim, 0:par_dim])
        A = np.diag(a)
        b = np.diag(M[0:par_dim, par_dim:2 * par_dim])
        B = np.diag(b)
        c = np.diag(M[par_dim:2 * par_dim, par_dim:2 * par_dim])
        C = np.diag(c)
        if (np.linalg.norm(A - M[0:par_dim, 0:par_dim]) < 1e-15 and
            np.linalg.norm(B - M[0:par_dim, par_dim:2*par_dim]) < 1e-15 and
            np.linalg.norm(C - M[par_dim:2*par_dim, par_dim:2 * par_dim]) < 1e-15):
            D = M[2 * par_dim:, 2 * par_dim:]
            if (np.linalg.matrix_rank(A) < A.shape[0]):
                a = a + np.ones(a.shape[0])*self.reg_coef
            if (np.linalg.matrix_rank(C) < C.shape[0]):
                c = c + np.ones(a.shape[0]) * self.reg_coef
            if (np.linalg.matrix_rank(D) < D.shape[0]):
                D = D + np.eye(D.shape[0]) * self.reg_coef
            Di = np.linalg.inv(D)
            e = 1. / (a - b / c * b)
            f = -e * b / c
            h = (np.ones(a.shape[0]) - f * b) / c
            dX = np.zeros(N.shape)
            for i in range(0, n_samples):
                N1 = N[:, i * n_dim:(i + 1) * n_dim]
                for gi in range(0, gm_num):
                    n_mu_gi = np.diag(N1[gi * n_dim:(gi + 1) * n_dim, 0:n_dim])
                    e_gi = e[gi * n_dim:(gi + 1) * n_dim]
                    n_cov_gi = np.diag(
                        N1[n_dim * gm_num + gi * n_dim:n_dim * gm_num + (gi + 1) * n_dim, 0:n_dim])
                    f_gi = f[gi * n_dim:(gi + 1) * n_dim]
                    h_gi = h[gi * n_dim:(gi + 1) * n_dim]
                    dX[gi * n_dim: (gi + 1) * n_dim, i * n_dim:(i + 1) * n_dim] = np.diag(
                        e_gi * n_mu_gi + f_gi * n_cov_gi)
                    dX[n_dim * gm_num + gi * n_dim: n_dim * gm_num + (gi + 1) * n_dim,
                    i * n_dim: (i + 1) * n_dim] = np.diag(f_gi * n_mu_gi + h_gi * n_cov_gi)
            dX[n_dim * 2 * gm_num:, :] = Di.dot(N[n_dim * 2 * gm_num:, :])
        else:
            M = M + self.reg_coef * np.eye(M.shape[0])
            dX = np.linalg.solve(M, N)
        return dX
    

    def perform(self, node, (X,), output_storage):
        gmm = mixture.GaussianMixture(covariance_type='diag',n_components=self.gm_num, max_iter=2000)
        gmm.fit(X.reshape((-1,self.ndim)))
        means = np.copy(gmm.means_).astype(np.float32)
        covars = np.copy(gmm.covariances_).astype(np.float32)
        weights = np.copy(gmm.weights_).astype(np.float32)
        dX = self.solve_lin_sys_for_gmm(X.flatten(),means.flatten(),covars.flatten(),weights.flatten())
        output_storage[0][0]=dX[0:dX.shape[0]-1, :].astype(np.float32)

class GMMOp(theano.Op):
    def __init__(self,gm_num,ndim):
        """
        fit gmm with diagonal covariances to input vectors
        input: vector[n_samples*n_dim] flattened
        output: vector means[gm_num*n_dim],covars[gm_num*n_dim],weights[gm_num], flattened
        """
        super(GMMOp, self).__init__()
        self.otypes = [T.fvector]
        self.itypes = [T.fvector]
        self.gm_num = gm_num
        self.ndim = ndim
        self.gmm = mixture.GaussianMixture(covariance_type='diag',n_components=self.gm_num, max_iter=2000)

    def perform(self, node, (X,), output_storage):       
        self.gmm.fit(X.reshape((-1,self.ndim)))
        means = self.gmm.means_.flatten()
        covars = self.gmm.covariances_.flatten()
        weights = self.gmm.weights_.flatten()
        output_storage[0][0] = np.concatenate((means,covars,weights)).astype(np.float32)
        
    def grad(self, (X,), output_grads):
        return [output_grads[0].dot(GMMGradOp(self.gm_num,self.ndim)(X))]
    
def get_gmm(X,gm_num,ndims):
    f = GMMOp(gm_num,ndims)(X.flatten())
    means = f[:gm_num*ndims].reshape((gm_num,ndims))
    covars = f[gm_num*ndims:2*gm_num*ndims].reshape((gm_num,ndims))
    weights = f[2*gm_num*ndims:]
    return means,covars,weights

In [74]:
def test_1_gmm(n_samp,dim):
    X = T.fmatrix("X")
    m1,c1,w1 =  get_gmm(X,1,dim)
    
    m2 = T.mean(X,axis=0).reshape((1,-1))
    c2 = (T.std(X,axis=0)+0.0001).reshape((1,-1))
    
    f = function([X],[m1,c1,w1,m2,c2])
    res = f(np.random.randn(n_samp,dim).astype(np.float32))
    
    print'dif mean,dif cov, w', np.abs(res[0]-res[3]).mean(),np.abs(res[1]-res[4]).mean(),np.abs(res[2]).sum()
    
    dm1 = gradient.jacobian(m1.flatten(),[X])
    dc1 = gradient.jacobian(c1.flatten(),[X])
    dw1 = gradient.jacobian(w1.flatten(),[X])
    dm2 = gradient.jacobian(m2.flatten(),[X])
    dc2 = gradient.jacobian(m2.flatten(),[X])
    
    df = function([X],dm1+dc1+dw1+dm2+dc2)
    res = df(np.random.randn(n_samp,dim).astype(np.float32))
    print'dif mean,dif cov, w', np.abs(res[0]-res[3]).mean(),np.abs(res[1]-res[4]).mean(),np.abs(res[2]).sum()

test_1_gmm(1000,10)

dif mean,dif cov, w 5.82077e-10 0.01034 1.0
dif mean,dif cov, w 1.39698e-14 0.000176987 0.0
